# 05. Transfer Learning with TensorFlow Part 2: Fine-tuning!

```
All code and text is handwritten by myself, as part of coding along to Daniel Bourke's series

https://github.com/mrdbourke
```

Previously, we leveraged *feature extraction* transfer learning to get better results than our own model got, with only 10% of the original data used. 

In this section we learn about another type of transfer learning, *fine-tuning*!

For this type of transfer learning we unfreeze the pre-trained model weights that are imported and tweak them to better suit our own data.

In *feature extraction transfer learning* generally only the top 1-3 layers of the pre-trained model are trained on our own data, but in *fine-tuning transfer learning* anywhere from 1 to all of the layers of the pre-trained model could be trained. 

![](https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/05-transfer-learning-feature-extraction-vs-fine-tuning.png)
*Feature extraction transfer learning vs. fine-tuning transfer learning. The main difference between the two is that in fine-tuning, more layers of the pre-trained model get unfrozen and tuned on custom data. This fine-tuning usually takes more data than feature extraction to be effective.*

## Specifics in this Notebook
* Introduction to fine-tuning
* Use the Keras Functional API
* Continue to use the smaller dataset
* Data augmentation to boost our training
* Running a series of experiments on the Food Vision Data
  - Model 0: transfer learning model using the Keras Functional API
  - Model 1: 
  - Model 2: a feature extraction transfer learning model on 10% of the data with data augmentation
  - Model 3: a fine-tuned transfer learning model on 10% of the data
  - Model 4: a fine-tuned transfer learning model on 100% of the data
* Introduction to ModelCheckpoint callback to save intermediate results from longer training sessions
* Comparing results with TensorBoard



In [1]:
# Confirm we are running on a GPU for faster results
!nvidia-smi

Tue Jun 28 18:05:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Creating helper functions

Helper functions are blocks of code that generally will be used frequently between projects. 
* e.g. a plotting function for a model's `history` object (in our case the `plot_loss_curves()` function).
* A good way to access these functions is through storing them in a helper script such as [`helper_functions.py`](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py). And them import the needed functionality.

In [2]:
# Get helper_functions.py from the course GitHub
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 

# Import the helper functions that are needed for this module
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

--2022-06-28 18:09:02--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-28 18:09:03 (103 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Using 10 Food classes

We will continue to use smaller data sets than in the first 3 modules

We will use the pretrained models within tf.keras.applications and also how to fine-tune them to our custom dataset.

We will use a new dataloader function `image_dataset_from_directory`.

We will be using the Keras Functional API to build our models. It's more flexible than using the tf.keras.Sequential() API that we used in the previous 4 modules.

In [3]:
# Get 10% of the 10 class data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip 

unzip_data("10_food_classes_10_percent.zip")

--2022-06-28 18:12:36--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 172.217.194.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  58.8MB/s    in 2.7s    

2022-06-28 18:12:39 (58.8 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [4]:
# Walk through the directories
walk_through_dir("10_food_classes_10_percent/")

There are 2 directories and 0 images in '10_food_classes_10_percent/'.
There are 10 directories and 0 images in '10_food_classes_10_percent/test'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/fried_rice'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/chicken_wings'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/hamburger'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/grilled_salmon'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/sushi'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/pizza'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ice_cream'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/steak'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/ramen'.
There are 0 directories and 250 images in '10_food_classes_10_percent/test/c

In [5]:
# Define training and test filpaths
# Create  training and test directories
train_dir = '10_food_classes_10_percent/train/'
test_dir = '10_food_classes_10_percent/test/'

We are switching to `image_data_from_directory` instead of `ImageDataGenerator`'s `flow_from_directory`. However it works in a similar manner and needs the same folder structure

One of the main benefits of using [`tf.keras.prepreprocessing.image_dataset_from_directory()`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) rather than `ImageDataGenerator` is that it creates a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) object rather than a generator. The main advantage of this is the `tf.data.Dataset` API is much more efficient (faster) than the `ImageDataGenerator` API which is paramount for larger datasets.


In [7]:
# Creat data inputs
import tensorflow as tf
IMG_SIZE=(224,224)
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode='categorical',
                                                                            batch_size=32)
test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode='categorical')

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [8]:
# Check the training data datatype
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

For the above:
- `(None, 224, 224, 3)` is the tensor shape of our images. `None=batch size`, `224=height and width`, `3=color channel`
- `(None, 10)` refers to the tensor shape of the labels where `None=batch size`, and `10=number of possible labels`

`batch_size` is `None` because this is oinly used during model training. Currently it is acting like a placeholder.

In [9]:
# Class names of the dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [11]:
# See example of batch of data using `take()` method
for images, labels in train_data_10_percent.take(1):
  print(images, labels)

tf.Tensor(
[[[[161.64285   177.64285   167.64285  ]
   [163.57143   179.57143   169.57143  ]
   [165.64285   181.64285   171.64285  ]
   ...
   [100.29079   103.29079    84.29079  ]
   [ 97.        100.         81.       ]
   [ 97.35714   100.35714    81.35714  ]]

  [[161.73979   177.73979   167.73979  ]
   [163.0051    179.0051    169.0051   ]
   [165.08673   181.08673   171.08673  ]
   ...
   [101.44383   104.44383    85.44383  ]
   [ 97.07144   100.07144    81.07144  ]
   [ 99.38269   102.38269    83.38269  ]]

  [[163.78572   179.78572   169.78572  ]
   [164.        180.        170.       ]
   [166.21428   182.21428   172.21428  ]
   ...
   [101.64279   104.64279    85.64279  ]
   [ 97.07144   100.07144    81.07144  ]
   [100.21943   103.21943    84.21943  ]]

  ...

  [[109.280624  118.63774   110.06633  ]
   [114.55615   123.91327   115.12756  ]
   [115.00003   124.49491   115.23472  ]
   ...
   [  6.          7.          2.       ]
   [  6.2704268   7.2704268   2.2704265]
   [ 